## 1.制造数据集

In [2]:
# 生成 y = Xw+b+噪声
import torch
# w: 是一个向量（权重），在这个例子中它是一个一维张量。
# b: 是一个标量（偏置项）。
# num_examples: 表示要生成的数据点数量。
def synthetic_data(w,b,num_examples):
    # 这行代码生成了一个形状为 (num_examples, len(w)) 的张量，其中每个元素都是从均值为0，标准差为1的正态分布中随机抽取的。
    x = torch.normal(0,1,(num_examples,len(w))) # 
    # 这里使用了矩阵乘法 (torch.matmul) 来计算特征 x 和权重 w 的乘积，然后加上偏置项 b。这个操作遵循线性回归模型的公式 y = Xw + b。
    y = torch.matmul(x,w) + b # 
    # 为了使数据更加真实，这里在 y 中添加了随机噪声。噪声是从均值为0，标准差为0.01的正态分布中抽取的，与 y 的形状相同，这样每个数据点都会添加一些随机误差。
    y+=torch.normal(0,0.01,y.shape)
    return x,y.reshape((-1,1))

# 参数w
true_w = torch.tensor([2,-4.3])
# 参数b
true_b = 4.2
features,labels = synthetic_data(true_w, true_b, 1000)
print(features)
print(labels)


tensor([[ 0.1266,  0.1258],
        [-0.0984, -0.3571],
        [-0.1183,  0.9281],
        ...,
        [ 0.7442, -2.1322],
        [ 1.0019, -1.0791],
        [ 0.0818, -0.1324]])
tensor([[ 3.9082e+00],
        [ 5.5323e+00],
        [-1.9414e-02],
        [ 8.7438e+00],
        [ 1.2093e+01],
        [-2.1559e+00],
        [ 6.0192e+00],
        [-9.2056e-01],
        [ 6.7591e+00],
        [ 2.2358e+00],
        [ 5.2470e+00],
        [-5.6480e-01],
        [ 2.3931e+00],
        [ 2.9224e-01],
        [ 3.9571e+00],
        [-5.8338e-01],
        [ 4.0826e+00],
        [ 5.6452e+00],
        [ 6.6305e+00],
        [ 7.6563e+00],
        [-2.4471e+00],
        [ 1.1301e+01],
        [ 2.2953e+00],
        [ 4.3983e+00],
        [ 5.8962e+00],
        [ 3.1461e+00],
        [-6.3415e-01],
        [ 6.3351e+00],
        [ 5.7367e+00],
        [ 8.0657e-01],
        [ 5.9051e+00],
        [ 2.4243e+00],
        [ 8.9170e+00],
        [ 4.7309e+00],
        [-1.4149e+00],
        [ 5.1

## 2.线性回归的简单实现
### 2.1数据创建

In [11]:
import numpy as np
import torch
from torch.utils import data

# 获取数据
true_w = torch.tensor([2,-4.3])
true_b = 4.2
features,labels = synthetic_data(true_w, true_b, 1000)
print(features)

tensor([[-1.5392, -0.3436],
        [ 0.8548, -1.2172],
        [-0.2436,  1.5250],
        ...,
        [ 1.4164, -0.8095],
        [-0.1740, -0.7856],
        [ 0.7990,  1.2201]])


### 2.2调用框架中现有API来读取数据

In [13]:
def load_array(data_arrays,batch_size,is_train=True):
    """构造一个pytorch数据迭代器"""
    # 这里使用 TensorDataset 类来将一组数据数组打包成一个数据集。*data_arrays 是一个 unpack 操作符，它会将传入的元组或列表解包为多个参数，所以如果 data_arrays 是一个包含两个张量的元组，比如 (features, labels)，那么这行代码相当于 dataset = data.TensorDataset(features, labels)。
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size=10
data_iter = load_array((features,labels),batch_size)
next(iter(data_iter))

[tensor([[-0.1500, -1.4906],
         [ 2.1675,  0.1806],
         [ 0.3004, -0.3093],
         [-1.2712,  0.0700],
         [ 0.9233, -0.8649],
         [ 0.2930, -0.3645],
         [ 0.1450, -0.0736],
         [-0.3287, -1.6242],
         [-0.9519, -0.7833],
         [-0.7147, -1.7744]]),
 tensor([[10.3118],
         [ 7.7527],
         [ 6.1273],
         [ 1.3418],
         [ 9.7652],
         [ 6.3629],
         [ 4.8070],
         [10.5297],
         [ 5.6826],
         [10.4025]])]

### 2.3使用框架预先定义好的层

In [15]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

### 2.4初始化模型参数

In [18]:
# [0]是第0层
net[0].weight.data.normal_(0,0.1)
net[0].bias.data.fill_(0)

tensor([0.])

### 2.5计算均方误差

In [19]:
loss = nn.MSELoss()

### 2.6SGD算法

In [21]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

### 2.7训练过程

In [32]:
epochs = 3
for epoch in range(epochs):
    for x,y in data_iter:
        l = loss(net(x),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    # 在每轮训练结束后，再次计算模型在完整数据集上的损失，并打印出来，以便监控训练进度和模型性能。
    l = loss(net(features),labels)
    print(f'epoch:{epoch+1} loss:{l}')
        

epoch:1 loss:0.00010324995673727244
epoch:2 loss:0.00010380881576566026
epoch:3 loss:0.0001035891255014576
